In [2]:
from azure.identity import DefaultAzureCredential,InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except  Exception as ex:
    credential = InteractiveBrowserCredential()

ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json
Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [ ]:
# When you compare and evaluate your machine learning models, you'll want to review more than just their performance metric. 
# Azure Machine Learning allows you to create responsible AI dashboard to explore how the model performs on different cohorts of the data. 
# Convert Csv Data into DataFrame
import pandas as pd

df_training = pd.read_csv('resposible_dashboard/train-data/diabetes.csv')
df_test = pd.read_csv('resposible_dashboard/test-data/diabetes-test.csv')

df_training.head()


,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,0,171,80,34,23,43.509726,1.213191,21,0
1,8,92,93,47,36,21.240576,0.158365,23,0
2,7,115,47,52,35,41.511523,0.079019,23,0
3,9,103,78,25,304,29.582192,1.282870,43,1
4,1,85,59,27,35,42.604536,0.549542,22,0


In [5]:
df_test.head()

,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1,78,41,33,311,50.796392,0.420804,24,0
1,0,116,92,16,184,18.603630,0.131156,22,0
2,8,171,42,29,160,35.482247,0.082671,22,1
3,3,108,63,45,297,49.375169,0.100979,46,1
4,8,153,99,15,41,35.062139,0.116191,22,1


In [ ]:
# Convert DataFrame into parquet(columnar storage)
import pyarrow as pa
import pyarrow.parquet as pq

table_training = pa.Table.from_pandas(df_training)
table_test = pa.Table.from_pandas(df_test)

pq.write_table(table_training, "resposible_dashboard/train-data/diabetes-training.parquet", version="1.0")
pq.write_table(table_test, "resposible_dashboard/test-data/diabetes-test.parquet", version="1.0")

In [16]:
# Create DataAssets
# Below is Path
train_data_path = "resposible_dashboard/train-data/"
test_data_path = "resposible_dashboard/test-data/"
data_version = "1"

In [17]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

input_train_data = "diabetes_train_mltable"
input_test_data = "diabetes_test_mltable"

try:
    train_data = ml_client.data.get(
       name = input_train_data,
       version = data_version
    )

    test_data = ml_client.data.get(
       name = input_test_data,
       version = data_version
    )
except Exception as ex:
    train_data = Data(
        path= train_data_path,
        type=AssetTypes.MLTABLE,
        description="RAI diabetes training data",
        name=input_train_data,
        version=data_version
    )

    ml_client.data.create_or_update(train_data)

    test_data = Data(
        path= test_data_path,
        type=AssetTypes.MLTABLE,
        description="RAI diabetes test data",
        name=input_test_data,
        version=data_version
    )
    
    ml_client.data.create_or_update(test_data)

Uploading train-data (0.7 MBs): 100%|██████████| 697376/697376 [00:00<00:00, 11338730.48it/s]


Uploading test-data (0.35 MBs): 100%|██████████| 352180/352180 [00:00<00:00, 7075116.91it/s]




In [19]:
# Build the Pipeline
registry_name = 'azureml'
ml_client_registry = MLClient(
    credential=credential,
    subscription_id=ml_client.subscription_id,
    resource_group_name=ml_client.resource_group_name,
    registry_name=registry_name,
)
print(ml_client_registry)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x786b6b5b32e0>,
         subscription_id=6c6683e9-e5fe-4038-8519-ce6ebec2ba15,
         resource_group_name=registry-builtin-prod-eastus-01,
         workspace_name=None)


In [21]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="resposible_dashboard/model",
    type=AssetTypes.MLFLOW_MODEL,
    name="local-mlflow-diabetes",
    description="Model created from local file.",
)
model = ml_client.models.create_or_update(file_model)

In [23]:
model_name = model.name
expected_model_id = f"{model_name}:1"
azureml_model_id = f"azureml:{expected_model_id}"
print(model_name)
print(expected_model_id)
print(azureml_model_id)

local-mlflow-diabetes
local-mlflow-diabetes:1
azureml:local-mlflow-diabetes:1


In [24]:
label = "latest"

rai_constructor_component = ml_client_registry.components.get(
    name="rai_tabular_insight_constructor", label=label
)

# we get latest version and use the same version for all components
version = rai_constructor_component.version
print("The current version of RAI built-in components is: " + version)

The current version of RAI built-in components is: 0.21.0


In [25]:
rai_erroranalysis_component = ml_client_registry.components.get(
    name="rai_tabular_erroranalysis", version=version
)

print("The current version of RAI Error-in components is: " + version)

rai_explanation_component = ml_client_registry.components.get(
    name="rai_tabular_explanation", version=version
)

print("The current version of RAI Tabular-in components is: " + version)

rai_gather_component = ml_client_registry.components.get(
    name="rai_tabular_insight_gather", version=version
)

print("The current version of RAI Gathe-in components is: " + version)

The current version of RAI Error-in components is: 0.21.0
The current version of RAI Tabular-in components is: 0.21.0
The current version of RAI Gathe-in components is: 0.21.0


In [27]:
from azure.ai.ml import Input,dsl
from azure.ai.ml.constants import AssetTypes

compute_name = "akacompute"

@dsl.pipeline(
    compute_name=compute_name,
    description="RAI insights on diabetes data",
    experiment_name=f"RAI_insights_{model_name}"
)

def rai_decision_pipeline(target_column_name,train_data,test_data):
    create_rai_job = rai_constructor_component(
        title="RAI dashboard diabetes",
        task_type="classification",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        train_dataset=train_data,
        test_dataset=test_data,
        target_column_name=target_column_name,
    )    

    create_rai_job.set_limits(timeout=300)

    # Add error analysis
    error_job = rai_erroranalysis_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
    )
    error_job.set_limits(timeout=300)

    # Add explanations
    explanation_job = rai_explanation_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        comment="add explanation", 
    )
    explanation_job.set_limits(timeout=300)

    # Combine everything
    rai_gather_job = rai_gather_component(
        constructor=create_rai_job.outputs.rai_insights_dashboard,
        insight_3=error_job.outputs.error_analysis,
        insight_4=explanation_job.outputs.explanation,
    )
    rai_gather_job.set_limits(timeout=300)

    rai_gather_job.outputs.dashboard.mode = "upload"

    return {
        "dashboard": rai_gather_job.outputs.dashboard,
    }


In [28]:
from azure.ai.ml import Input
target_feature = "Diabetic"

diabetes_train_pq = Input(
    type="mltable",
    path=f"azureml:{input_train_data}:{data_version}",
    mode="download",
)
diabetes_test_pq = Input(
    type="mltable",
    path=f"azureml:{input_test_data}:{data_version}",
    mode="download",
)

In [31]:
import uuid
from azure.ai.ml import Output

# Pipeline to construct the RAI Insights
insights_pipeline_job = rai_decision_pipeline(
    target_column_name="Diabetic",
    train_data=diabetes_train_pq,
    test_data=diabetes_test_pq,
)

insights_pipeline_job.settings.default_compute = "akacompute"

# Workaround to enable the download
rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)


In [32]:
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display
import time

def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job


# This is the actual submission
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Pipeline job can be accessed in the following URL:


Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Completed
